# NEIGHBORHOOD AND STREET DATA FOR RIO DE JANEIRO CITY - Data type conversion

#### Set directory

In [1]:
cd ../

C:\Users\luisr\Desktop\Repositories\Data Science Projects\Hackaton COR IV - Centro de Operações do RJ\INCUBAÇÃO


#### Install dependencies

In [182]:
!pip install geomet

#### Import modules

In [183]:
import json, pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from geomet import wkt

from Modulos.geojson_conversion import linestring_geojson, polygon_geojson, save_json

# import os, json, pandas as pd, numpy as np, requests, pickle, pymongo
# from datetime import datetime, timezone
# from sklearn.preprocessing import MinMaxScaler as mms

---
# Functions

### Build geojson objects

In [214]:
def geojson_obj(features):
    return {
        "type": "FeatureCollection",
        "features": features
    }

def feature_obj(geometry):
    return {
        'type': 'Feature',
        'geometry': geometry
    }

def add_props(features, df):
    for feat, (idx, row) in zip(features, df.iterrows()):
        feat['properties'] = row.to_dict()
    return features

### Convert wkt dataframe to geojson

In [217]:
def wkt_geojson(df, wkt_col, props):
    geometries = df[wkt_col].apply(wkt.loads)
    features = geometries.apply(feature_obj)
    features = add_props(features.tolist(), df[props])
    return geojson_obj(features)

### BigQuery request function

In [3]:
project_id = 'pluvia-360323'
google_credentials = '../../../Apps/Python/bolsao-api/credentials/pluvia-360323-eae2907a9c98.json'
google_credentials = service_account.Credentials.from_service_account_file(google_credentials)

# WHERE data_particao >= "{}"
# ORDER BY data_particao DESC, horario DESC

def bigquery_request(query, params=[], google_credentials=None):
    client = bigquery.Client(credentials=google_credentials)
    query_job = client.query(query.format(**params))
    return pd.DataFrame(list(map(dict, query_job.result())))

---
## Get neighborhood dataset

In [4]:
query = '''
SELECT * FROM `datario.dados_mestres.bairro`
'''

bairros = bigquery_request(query, {}, google_credentials)

bairros.head()

,id_bairro,nome,id_area_planejamento,id_regiao_planejamento,nome_regiao_planejamento,id_regiao_administrativa,nome_regiao_administrativa,subprefeitura,area,perimetro,geometria_wkt,geometria
0,2,Gamboa,1,1.1,Centro,1,PORTUARIA,Centro,1.112903e+06,4612.827414,POLYGON ((-43.18787320828573 -22.8931089710106...,"POLYGON((-43.1878732082857 -22.8931089710107, ..."
1,1,Saúde,1,1.1,Centro,1,PORTUARIA,Centro,3.638176e+05,2646.216986,POLYGON ((-43.181109755485245 -22.895417528115...,"POLYGON((-43.1811097554853 -22.8954175281158, ..."
2,3,Santo Cristo,1,1.1,Centro,1,PORTUARIA,Centro,1.684721e+06,6743.218804,POLYGON ((-43.19445618134857 -22.9033652388182...,"POLYGON((-43.1944561813486 -22.9033652388182, ..."
3,4,Caju,1,1.1,Centro,1,PORTUARIA,Centro,5.347481e+06,19800.495942,MULTIPOLYGON (((-43.22518208553896 -22.8746396...,None
4,5,Centro,1,1.1,Centro,2,CENTRO,Centro,5.424739e+06,22846.214306,MULTIPOLYGON (((-43.178288869275505 -22.892566...,MULTIPOLYGON(((-43.1657633602567 -22.897028697...


#### Convert from wkt to geojson

In [218]:
df =  bairros
wkt_col = 'geometria_wkt'
props = bairros.columns.drop(['geometria', 'geometria_wkt'])

bairros_geojson = wkt_geojson(df, wkt_col, props)

save_json(bairros_geojson, 'Dados/Mestres/bairro.geojson')

Done!


## Get street dataset

In [254]:
hierarquias = ('Estrutural', 'Arterial primária', 'Arterial secundária')

query = '''
SELECT * FROM `datario.dados_mestres.logradouro` WHERE hierarquia IN {}
'''.format(hierarquias)

vias = bigquery_request(query, {}, google_credentials)

vias.head()

,id_logradouro,nome,nome_parcial,nome_completo,id_trecho,id_situacao_trecho,situacao_trecho,id_tipo,tipo,id_bairro,inicio_numero_porta_par,final_numero_porta_par,inicio_numero_porta_impar,final_numero_porta_impar,hierarquia,id_chavegeo,geometry
0,11080,,Álvaro Alberto,Rua Álvaro Alberto,208514,1,Ativo,41,Rua,,0,0,0,0,Estrutural,I7P3HM85,LINESTRING(-43.6847356687256 -22.9162453890271...
1,11080,,Álvaro Alberto,Rua Álvaro Alberto,208515,1,Ativo,41,Rua,,0,0,0,0,Estrutural,I7P3HM85,LINESTRING(-43.6841093726041 -22.9159378031191...
2,11676,,Manso,Rua Padre Manso,303301,6,,41,Rua,,12,298,29,29,Arterial primária,KYZBYI16,LINESTRING(-43.3375655501557 -22.8824684156819...
3,11676,,Manso,Rua Padre Manso,255414,1,Ativo,41,Rua,,0,0,0,0,Arterial primária,P61C4146,LINESTRING(-43.3373271704989 -22.8791011064349...
4,11676,,Manso,Rua Padre Manso,255415,1,Ativo,41,Rua,,0,0,0,0,Arterial primária,P61C4146,"LINESTRING(-43.337326270377 -22.8790936284571,..."


#### Check filter result

In [257]:
vias['hierarquia'].value_counts().to_frame()

,hierarquia
Arterial secundária,3119
Estrutural,2891
Arterial primária,2649


#### Convert from wkt to geojson

In [258]:
wkt_col = 'geometry'
props = vias.columns.drop('geometry')

vias_geojson = wkt_geojson(vias, wkt_col, props)

save_json(vias_geojson, 'Dados/Mestres/logradouro.geojson')

Done!
